In [1]:
import json
import random
import string

from vllm import LLM
from vllm.sampling_params import SamplingParams


INFO 06-02 16:31:18 [__init__.py:239] Automatically detected platform cuda.


In [2]:
model_name = "/mnt/petrelfs/songmingyang/songmingyang/model/mm/Qwen2.5-VL-3B-Instruct"
# or switch to "mistralai/Mistral-Nemo-Instruct-2407"
# or "mistralai/Mistral-Large-Instruct-2407"
# or any other mistral model with function calling ability

sampling_params = SamplingParams(max_tokens=8192, temperature=0.0)
llm = LLM(
    model=model_name,
)

INFO 06-02 16:31:39 [config.py:585] This model supports multiple tasks: {'score', 'reward', 'embed', 'classify', 'generate'}. Defaulting to 'generate'.
INFO 06-02 16:31:39 [config.py:1697] Chunked prefill is enabled with max_num_batched_tokens=8192.
INFO 06-02 16:31:41 [core.py:54] Initializing a V1 LLM engine (v0.8.2) with config: model='/mnt/petrelfs/songmingyang/songmingyang/model/mm/Qwen2.5-VL-3B-Instruct', speculative_config=None, tokenizer='/mnt/petrelfs/songmingyang/songmingyang/model/mm/Qwen2.5-VL-3B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=128000, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar', reasoning_backend=None

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


INFO 06-02 16:31:43 [gpu_model_runner.py:1174] Starting to load model /mnt/petrelfs/songmingyang/songmingyang/model/mm/Qwen2.5-VL-3B-Instruct...
INFO 06-02 16:31:44 [config.py:3243] cudagraph sizes specified by model runner [1, 2, 4, 8, 16, 24, 32, 40, 48, 56, 64, 72, 80, 88, 96, 104, 112, 120, 128, 136, 144, 152, 160, 168, 176, 184, 192, 200, 208, 216, 224, 232, 240, 248, 256, 264, 272, 280, 288, 296, 304, 312, 320, 328, 336, 344, 352, 360, 368, 376, 384, 392, 400, 408, 416, 424, 432, 440, 448, 456, 464, 472, 480, 488, 496, 504, 512] is overridden by config [512, 384, 256, 128, 4, 2, 1, 392, 264, 136, 8, 400, 272, 144, 16, 408, 280, 152, 24, 416, 288, 160, 32, 424, 296, 168, 40, 432, 304, 176, 48, 440, 312, 184, 56, 448, 320, 192, 64, 456, 328, 200, 72, 464, 336, 208, 80, 472, 344, 216, 88, 120, 480, 352, 248, 224, 96, 488, 504, 360, 232, 104, 496, 368, 240, 112, 376]
WARNING 06-02 16:31:44 [topk_topp_sampler.py:63] FlashInfer is not available. Falling back to the PyTorch-native imple

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


INFO 06-02 16:31:54 [loader.py:447] Loading weights took 9.47 seconds
INFO 06-02 16:31:54 [gpu_model_runner.py:1186] Model loading took 7.1557 GB and 10.179992 seconds
INFO 06-02 16:31:54 [gpu_model_runner.py:1456] Encoder cache will be initialized with a budget of 98304 tokens, and profiled with 1 video items of the maximum feature size.
INFO 06-02 16:32:15 [backends.py:415] Using cache directory: /mnt/petrelfs/songmingyang/.cache/vllm/torch_compile_cache/ab008d54b6/rank_0_0 for vLLM's torch.compile
INFO 06-02 16:32:15 [backends.py:425] Dynamo bytecode transform time: 10.71 s
INFO 06-02 16:32:16 [backends.py:115] Directly load the compiled graph for shape None from the cache
INFO 06-02 16:32:26 [monitor.py:33] torch.compile takes 10.71 s in total
INFO 06-02 16:32:27 [kv_cache_utils.py:566] GPU KV cache size: 1,679,280 tokens
INFO 06-02 16:32:27 [kv_cache_utils.py:569] Maximum concurrency for 128,000 tokens per request: 13.12x
INFO 06-02 16:32:49 [gpu_model_runner.py:1534] Graph captur

In [11]:

def generate_random_id(length=9):
    characters = string.ascii_letters + string.digits
    random_id = "".join(random.choice(characters) for _ in range(length))
    return random_id


# simulate an API that can be called
def get_current_weather(city: str, state: str, unit: "str"):
    return (
        f"The weather in {city}, {state} is 85 degrees {unit}. It is "
        "partly cloudly, with highs in the 90's."
    )


tool_functions = {"get_current_weather": get_current_weather}

tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {
                        "type": "string",
                        "description": "The city to find the weather for, e.g. 'San Francisco'",
                    },
                    "state": {
                        "type": "string",
                        "description": "the two-letter abbreviation for the state that the city is"
                        " in, e.g. 'CA' which would mean 'California'",
                    },
                    "unit": {
                        "type": "string",
                        "description": "The unit to fetch the temperature in",
                        "enum": ["celsius", "fahrenheit"],
                    },
                },
                "required": ["city", "state", "unit"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_current_weather2",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {
                        "type": "string",
                        "description": "The city to find the weather for, e.g. 'San Francisco'",
                    },
                    "state": {
                        "type": "string",
                        "description": "the two-letter abbreviation for the state that the city is"
                        " in, e.g. 'CA' which would mean 'California'",
                    },
                    "unit": {
                        "type": "string",
                        "description": "The unit to fetch the temperature in",
                        "enum": ["celsius", "fahrenheit"],
                    },
                },
                "required": ["city", "state", "unit"],
            },
        },
    }
]

messages = [
    {
        "role": "user",
        "content": "Can you tell me what the temperate will be in Dallas, in fahrenheit?",
    }
]

outputs = llm.chat(messages, sampling_params=sampling_params, tools=tools)
output = outputs[0].outputs[0].text.strip()
output

Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  3.72it/s, est. speed input: 1595.26 toks/s, output: 126.72 toks/s]


'<tool_call>\n{"name": "get_current_weather", "arguments": {"city": "Dallas", "state": "TX", "unit": "fahrenheit"}}\n</tool_call>'

In [12]:
json_str = output.split("<tool_call>")[-1].split("</tool_call>")[0].strip()
json_str

'{"name": "get_current_weather", "arguments": {"city": "Dallas", "state": "TX", "unit": "fahrenheit"}}'

In [13]:

# append the assistant message
messages.append(
    {
        "role": "assistant",
        "content": output,
    }
)

# let's now actually parse and execute the model's output simulating an API call by using the
# above defined function
json_str = output.split("<tool_call>")[-1].split("</tool_call>")[0].strip()
tool_calls = json.loads(json_str)
tool_answers = [
    tool_functions[tool_calls["name"]](**tool_calls["arguments"]) for call in tool_calls
]

# append the answer as a tool message and let the LLM give you an answer
messages.append(
    {
        "role": "tool",
        "content": "\n\n".join(tool_answers),
        "tool_call_id": generate_random_id(),
    }
)

outputs = llm.chat(messages, sampling_params, tools=tools)

print(outputs[0].outputs[0].text.strip())

Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  4.02it/s, est. speed input: 2161.69 toks/s, output: 124.78 toks/s]

The current temperature in Dallas, TX is 85 degrees Fahrenheit. The weather is partly cloudy, with expected highs in the 90's.


In [14]:
tool_answers

["The weather in Dallas, TX is 85 degrees fahrenheit. It is partly cloudly, with highs in the 90's.",
 "The weather in Dallas, TX is 85 degrees fahrenheit. It is partly cloudly, with highs in the 90's."]

In [15]:
tool_calls

{'name': 'get_current_weather',
 'arguments': {'city': 'Dallas', 'state': 'TX', 'unit': 'fahrenheit'}}

In [16]:
outputs

[RequestOutput(request_id=4, prompt='<|im_start|>system\nYou are a helpful assistant.\n\n# Tools\n\nYou may call one or more functions to assist with the user query.\n\nYou are provided with function signatures within <tools></tools> XML tags:\n<tools>\n{"type": "function", "function": {"name": "get_current_weather", "description": "Get the current weather in a given location", "parameters": {"type": "object", "properties": {"city": {"type": "string", "description": "The city to find the weather for, e.g. \'San Francisco\'"}, "state": {"type": "string", "description": "the two-letter abbreviation for the state that the city is in, e.g. \'CA\' which would mean \'California\'"}, "unit": {"type": "string", "description": "The unit to fetch the temperature in", "enum": ["celsius", "fahrenheit"]}}, "required": ["city", "state", "unit"]}}}\n{"type": "function", "function": {"name": "get_current_weather2", "description": "Get the current weather in a given location", "parameters": {"type": "o